In [83]:
#2021 05 17, 18, 19 Создание отчета план - факт по БДДС. Промтех. Вологорад. ЗП



In [84]:
import os
os.getcwd()

'c:\\Reports'

In [85]:
#пути

os.chdir('c:\\Reports')
os.getcwd()

'c:\\Reports'

In [86]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, date, timedelta
import xlsxwriter
import openpyxl
import re

import pymssql

from copy import deepcopy

import datetime as dt


In [87]:
import matplotlib.pyplot as plt
import seaborn as sns

In [88]:
pd.set_option('display.float_format', '{:.2f}'.format)
# pd.set_option('display.max_rows', 100)

In [89]:
# подлючение к базе mssql
conn = pymssql.connect(server='XXX', user='XXX', password='XXX', database='XXX', charset='XXX')
cursor = conn.cursor()  

cursor.execute('SELECT * FROM uld_mtpromtehbuh.details')  

row = cursor.fetchone()

desc = cursor.description
row = cursor.fetchall()


# вывод названия столбцов по новому:
columns = [item[0] for item in cursor.description]


df0 = pd.DataFrame(row, columns=columns)

# 2021 10 20 загружает 8,2 сек
# 2021 10 28 загружает 9,5 сек

In [90]:
df0.tail()

,id,Depatment,Doc_id,date,SD,PodrDt,SKD1,SKD1_cod,SKD1_tip,SKD2,...,SKK2,SKK2_cod,SKK2_tip,SKK3,SKK3_cod,SKK3_tip,Summa,Valuta,ValSumma,Kol
169943,2213705,"ООО ""ПРОМТЕХ""",a859103c-52a3-11ec-bdd4-00155d1e1e08,2021-12-01,26,Административно-хозяйственный отдел,"26.25 Канцтовары, хозтовары, АХО",4.25,Статьи затрат,,...,Склад №19 Административно-хозяйственный,,Склады,,,,0.00,,0.00,36.00
169944,2213706,"ООО ""ПРОМТЕХ""",35d098f1-52a6-11ec-bdd4-00155d1e1e08,2021-12-01,90.03,Коммерческий отдел,,,Номенклатурные группы,20%,...,,,,,,,1936.67,,0.00,0.00
169945,2213707,"ООО ""ПРОМТЕХ""",35d098f1-52a6-11ec-bdd4-00155d1e1e08,2021-12-01,62.01,Коммерческий отдел,Агарков Александр Юрьевич,561403426638,Контрагенты,Продажа оросительных насадок б/у № 17/374 от 2...,...,20%,,Ставки НДС,Оросительные насадки б/у,УП-00007521,Номенклатура,11620.00,,0.00,1660.00
169946,2213708,"ООО ""ПРОМТЕХ""",35d098f1-52a6-11ec-bdd4-00155d1e1e08,2021-12-01,91.02,Коммерческий отдел,,,Прочие доходы и расходы,Оросительные насадки б/у,...,Основной,,Склады,,,,0.00,,0.00,1660.00
169947,2213709,"ООО ""ПРОМТЕХ""",57ae59a3-52a6-11ec-bdd4-00155d1e1e08,2021-12-01,10.09,"ОБОСОБЛЕННОЕ ПОДРАЗДЕЛЕНИЕ ""ПРОМТЕХ - ВОЛГОГРАД""",Оросительные насадки б/у,УП-00007521,Номенклатура,Основной,...,Оросительные насадки б/у,УП-00007521,Реализуемые активы,,,,9677.80,,0.00,1660.00


In [ ]:
print(df0['Depatment'].unique())

#Создаю дата фрейм только по Промтех

df0 = df0[df0['Depatment'] == 'ООО "ПРОМТЕХ"']

In [92]:
# Отбор статей движения ден средств
# df = df0[(df0['SKD1_tip']=='Статьи движения денежных средств') | (df0['SKD2_tip']=='Статьи движения денежных средств') | (df0['SKK1_tip']=='Статьи движения денежных средств') | (df0['SKK2_tip']=='Статьи движения денежных средств')]

# отбор только по счетам

df = df0[(df0['SD']=='50') | (df0['SD']=='51') | (df0['SD']=='52') | (df0['SK']=='50') | (df0['SK']=='51') | (df0['SK']=='52')]

In [93]:
#Столбец со статьями ДС

# перешел на .loc
df['Статья'] = ''

df.loc[df['SD']=='50', 'Статья'] = df['SKD1']
df.loc[df['SD']=='51', 'Статья'] = df['SKD2']
df.loc[df['SD']=='52', 'Статья'] = df['SKD2']
df.loc[df['SK']=='50', 'Статья'] = df['SKK1']
df.loc[df['SK']=='51', 'Статья'] = df['SKK2']
df.loc[df['SK']=='52', 'Статья'] = df['SKK2']

C:\Users\SMOLEN~1\AppData\Local\Temp\45/ipykernel_13044/2991043830.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Статья'] = ''
C:\Users\Smolenyshev_OV\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [94]:
# df['Статья'].str[:3].unique()

In [95]:
#Столбец Статья2 с вгрегированными группами статей

df['Группа'] = ''

df.loc[df['Статья'].str[:1]=='1', 'Группа'] = '1. Приток от опер-ной деятельности'
df.loc[df['Статья'].str[:3]=='2.1', 'Группа'] = '2.1 Расходы на персонал' 
df.loc[df['Статья'].str[:3]=='2.2', 'Группа'] = '2.2 Материалы'
df.loc[df['Статья'].str[:3]=='2.3', 'Группа'] = '2.3 Работы и услуги'
df.loc[df['Статья'].str[:3]=='2.4', 'Группа'] = '2.4 Страхование'
df.loc[df['Статья'].str[:3]=='2.5', 'Группа'] = '2.5 Налоги, сборы и платежи'
df.loc[df['Статья'].str[:3]=='2.6', 'Группа'] = '2.6 Прочие платежи'
df.loc[df['Статья'].str[:1]=='3', 'Группа'] = '3. Приток от инвест-ной деятельности'
df.loc[df['Статья'].str[:1]=='4', 'Группа'] = '4. Отток от инвест-ной деятельности'
df.loc[df['Статья'].str[:1]=='5', 'Группа'] = '5. Приток от фин-ной деятельности'
df.loc[df['Статья'].str[:1]=='6', 'Группа'] = '6. Отток от фин-ной деятельности'
df.loc[df['Статья'].str[:1]=='7', 'Группа'] = '7. Внутренние перемещения'
df.loc[df['Статья'].str[:7]=='Возврат', 'Группа'] = '9. Возврат'

C:\Users\SMOLEN~1\AppData\Local\Temp\45/ipykernel_13044/1203533715.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Группа'] = ''
C:\Users\Smolenyshev_OV\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [96]:
# df['Группа'].unique()



In [97]:
#удаляю знак \t:

df['Статья'].replace(to_replace="\t", value=' ', inplace=True, limit=None, regex=True)

# df['Статья'].unique()

C:\Users\Smolenyshev_OV\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [98]:
# дату в нормальный формат

df['date'] = df['date'].apply(pd.to_datetime).copy()

C:\Users\SMOLEN~1\AppData\Local\Temp\45/ipykernel_13044/2546626742.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].apply(pd.to_datetime).copy()


In [99]:
# нахождение крайней даты
lastday = df['date'].sort_values(ascending=True).tail(1).astype('datetime64[ns]')

#
print('lastday: ', lastday)
print('-------')

#привоэу в более менее удобный формат для выдачи в текст заголовка
lastday2 = lastday.astype('str').values

print('lastday2: ', lastday2)

lastday:  169936   2021-12-01
Name: date, dtype: datetime64[ns]
-------
lastday2:  ['2021-12-01']


In [100]:
lastday

169936   2021-12-01
Name: date, dtype: datetime64[ns]

In [101]:
# сегодня

today = pd.to_datetime('today')

today

Timestamp('2021-12-02 16:15:58.691809')

In [102]:
# сегодня в номальный вид

today2 = dt.datetime.today().strftime("%Y-%m-%d %H:%M")

today2

'2021-12-02 16:15'

In [103]:
# эксперементирую

today3 = pd.to_datetime(today2)

today3

Timestamp('2021-12-02 16:15:00')

In [104]:
# библиотека для работы с датами и днями
from pandas.tseries.offsets import MonthEnd

In [105]:
# последний день текущего месяца

dem = pd.to_datetime(today2)+MonthEnd(1)

dem

Timestamp('2021-12-31 16:15:00')

In [106]:
dem2 = dem.strftime('%Y-%m-%d')

dem2

'2021-12-31'

In [107]:
# Отбор только по периоду


df = df[(df['date']>= '2021-01-01') & (df['date']<= dem)]

In [108]:
# стролбцы с периодами
df['Год'] = df['date'].dt.year.astype('Int64').values
df['Месяц'] = df['date'].dt.month.astype('Int64').values
df['День'] = df['date'].dt.day.astype('Int64').values

In [109]:
#только год и месяц
df['YM'] = df['date'].dt.to_period('M')

In [110]:
# Числовое поле в тыс.
df['ФАКТ'] = pd.to_numeric(df["Summa"]/1000).round(2)




In [111]:
#Приток +, отток -С Лучше делать через .loc

# перешел на .lov
df['Факт'] = ''

df.loc[df['SD']=='50', 'Ф-т'] = df['ФАКТ']
df.loc[df['SD']=='51', 'Ф-т'] = df['ФАКТ']
df.loc[df['SD']=='52', 'Ф-т'] = df['ФАКТ']
df.loc[df['SK']=='50', 'Ф-т'] = df['ФАКТ']*-1
df.loc[df['SK']=='51', 'Ф-т'] = df['ФАКТ']*-1
df.loc[df['SK']=='52', 'Ф-т'] = df['ФАКТ']*-1


In [112]:
# округление
df['Факт'] = pd.to_numeric(df['Ф-т']).round(2)

In [113]:
# выбираю только нужные столбцы
# df = df[['date', 'YM', 'Год','Месяц','День', 'SD', 'SK', 'Статья', 'Статья2', 'ФАКТ', 'Ф-т']]

df = df[['date', 'YM', 'Год', 'Месяц', 'День', 'SD', 'SK', 'Группа', 'Статья', 'Факт', 'SKD1', 'SKD2', 'SKD3', 'SKK1', 'SKK2', 'SKK3']]

In [114]:
# df только по притоку ДС от опер и инвест деят-ти

df_pr = df[(df['Группа']=='1. Приток от опер-ной деятельности')  | (df['Группа']=='3. Приток от инвест-ной деятельности')]

df_pr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 799 entries, 94670 to 169850
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    799 non-null    datetime64[ns]
 1   YM      799 non-null    period[M]     
 2   Год     799 non-null    Int64         
 3   Месяц   799 non-null    Int64         
 4   День    799 non-null    Int64         
 5   SD      799 non-null    object        
 6   SK      799 non-null    object        
 7   Группа  799 non-null    object        
 8   Статья  799 non-null    object        
 9   Факт    799 non-null    float64       
 10  SKD1    799 non-null    object        
 11  SKD2    799 non-null    object        
 12  SKD3    799 non-null    object        
 13  SKK1    799 non-null    object        
 14  SKK2    799 non-null    object        
 15  SKK3    799 non-null    object        
dtypes: Int64(3), datetime64[ns](1), float64(1), object(10), period[M](1)
memory usage: 108.5+ K

In [115]:
# сводная табл по ОДДС по Статьям и Группам

pt1 = pd.pivot_table(df, index = ['Группа', 'Статья'], values = ['Факт'], columns = ['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt1

Факт  \
Год                                                                                      2021   
Месяц                                                                                       1   
Группа                             Статья                                                       
1. Приток от опер-ной деятельности 1.2.1 Поступления от аренды помещений                        
                                   1.2.3 Поступления от прочей аренды                   33.42   
                                   1.3.1 Реализация электроэнергии                              
                                   1.3.3 Реализация услуг железнодорожного транспорта 1590.22   
                                   1.3.7 Реализация услуг по водоснабжению                      
...                                                                                       ...   
6. Отток от фин-ной деятельности   6.10.1.4 Транспортные средства (Финансовая арен...           
7. Внутренние перемещения          7.1 Внутренние перемещения по расчетному счету        0.00   
9. Возврат                         Возврат денежных средств от поставщика                       
                                   Возврат оплаты клиенту                                       
All                                                                                   9065.04   

                                                                                                \
Год                                                                                              
Месяц                                                                                        2   
Группа                             Статья                                                        
1. Приток от опер-ной деятельности 1.2.1 Поступления от аренды помещений                  2.00   
                                   1.2.3 Поступления от прочей аренды                    35.18   
                                   1.3.1 Реализация электроэнергии                               
                                   1.3.3 Реализация услуг железнодорожного транспорта  1255.31   
                                   1.3.7 Реализация услуг по водоснабжению                       
...                                                                                        ...   
6. Отток от фин-ной деятельности   6.10.1.4 Транспортные средства (Финансовая арен...   -36.57   
7. Внутренние перемещения          7.1 Внутренние перемещения по расчетному счету         0.00   
9. Возврат                         Возврат денежных средств от поставщика                        
                                   Возврат оплаты клиенту                                        
All                                                                                   26964.28   

                                                                                                 \
Год                                                                                               
Месяц                                                                                         3   
Группа                             Статья                                                         
1. Приток от опер-ной деятельности 1.2.1 Поступления от аренды помещений                   2.00   
                                   1.2.3 Поступления от прочей аренды                     34.18   
                                   1.3.1 Реализация электроэнергии                                
                                   1.3.3 Реализация услуг железнодорожного транспорта   1314.76   
                                   1.3.7 Реализация услуг по водоснабжению                        
...                                                                                         ...   
6. Отток от фин-ной деятельности   6.10.1.4 Транспортные средства (Финансовая арен...  -1220.97   
7. Внутренние перемещения          7.1 Внутренние перемещения по расчетно

In [116]:
# сводная табл по ОДДС только по группам

pt2 = pd.pivot_table(df, index = ['Группа'], values = ['Факт'], columns = ['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt2

Факт                                \
Год                                       2021                                 
Месяц                                        1         2         3         4   
Группа                                                                         
1. Приток от опер-ной деятельности    25463.82  67131.00  87512.77  84189.05   
2.1 Расходы на персонал              -13981.78 -18359.88 -23099.06 -25230.26   
2.2 Материалы                        -10496.98  -7670.10  -9957.80  -8021.01   
2.3 Работы и услуги                   -1216.59  -4102.68 -11813.46  -7293.61   
2.4 Страхование                                            -185.61   -320.17   
2.5 Налоги, сборы и платежи           -2152.70  -1637.33 -39037.98 -15288.22   
2.6 Прочие платежи                    -6213.31 -10287.82  -7475.40  -9346.28   
3. Приток от инвест-ной деятельности   3580.78  13288.14  21950.30  13580.50   
4. Отток от инвест-ной деятельности    -430.83  -7360.48 -12981.23 -33437.64   
5. Приток от фин-ной деятельности     17012.63             7000.00  50000.00   
6. Отток от фин-ной деятельности      -2500.00  -4036.57 -22220.97  -4611.77   
7. Внутренние перемещения                 0.00      0.00      0.00      0.00   
9. Возврат                                                   75.00             
All                                    9065.04  26964.28 -10233.44  44220.59   

                                                                              \
Год                                                                            
Месяц                                        5         6         7         8   
Группа                                                                         
1. Приток от опер-ной деятельности    54991.34  99807.97  66941.94  69698.56   
2.1 Расходы на персонал              -17242.91 -19609.19 -18166.57 -15846.63   
2.2 Материалы                         -4272.70  -4738.89  -3935.35  -4998.96   
2.3 Работы и услуги                   -5649.86 -11887.05 -10938.56 -27124.49   
2.4 Страхование                        -143.27   -282.63   -186.64    -58.44   
2.5 Налоги, сборы и платежи           -5920.17  -5995.15  -8812.03  -8935.36   
2.6 Прочие платежи                    -7873.66 -16567.82  -8999.66 -11530.45   
3. Приток от инвест-ной деятельности   7713.63  16801.95   2189.60    690.00   
4. Отток от инвест-ной деятельности  -10641.48 -10688.61 -18785.10  -6065.57   
5. Приток от фин-ной деятельности               50011.99                       
6. Отток от фин-ной деятельности      -1683.67 -70032.61  -8084.25  -7504.08   
7. Внутренние перемещения                 0.00      0.00                0.00   
9. Возврат                                        -22.18               22.82   
All                                    9277.25  26797.78  -8776.62 -11652.60   

                                                                             \
Год                                                                           
Месяц                                        9        10        11       12   
Группа                                                                        
1. Приток от опер-ной деятельности    95666.35  50863.14  48345.54            
2.1 Расходы на персонал              -14586.13 -15236.57 -13568.25            
2.2 Материалы                         -4829.13  -5264.12  -5229.53            
2.3 Работы и услуги                  -12340.57 -36040.77 -22654.77            
2.4 Страхование                                            -101.81            
2.5 Налоги, сборы и платежи           -8653.86 -14456.67  -7834.36            
2.6 Прочие платежи                    -8716.11  -9077.70 -11257.48    -2.00   
3. Приток от инвест-ной деятельности   6412.00   2730.00   1360.00            
4. Отток от инвест-ной деятельности  -19532.64 -16886.45 -19837.63            
5. Приток от фин-ной деятельности                                   1169.53   
6. Отток от фин-ной деятельности      -8651.38 -11382.

In [117]:
# сводная табл по притоку ДС по опер и иныест деят по Статьям, Группам и контрагекнтам

pt3 = pd.pivot_table(df_pr, index = ['Группа', 'Статья', 'SKK1'], values = ['Факт'], columns = ['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt3

Факт  \
Год                                                                                                                        2021   
Месяц                                                                                                                         1   
Группа                               Статья                                             SKK1                                      
1. Приток от опер-ной деятельности   1.2.1 Поступления от аренды помещений              Алифанов Роман Игоревич                   
                                                                                        ВЫМПЕЛКОМ ПАО                             
                                                                                        ПРОМОСФЕРА ООО                            
                                                                                        ТТК-СВЯЗЬ ООО                             
                                     1.2.3 Поступления от прочей аренды                 ВЫМПЕЛКОМ ПАО                     33.42   
...                                                                                                                         ...   
3. Приток от инвест-ной деятельности 3.1.3 Транспортные средства (Выручка от продажи... Филлипенков Антон Андреевич ЧЛ            
                                                                                        ЦЕНТРУС 1520 ООО                          
                                                                                        Шикерханов Гаджибала Рафикович            
                                     3.1.4 Инженерные сети и оборудование (Выручка о... РЕГИОН ПЛЮС ООО                           
All                                                                                                                    29044.60   

                                                                                                                                 \
Год                                                                                                                               
Месяц                                                                                                                         2   
Группа                               Статья                                             SKK1                                      
1. Приток от опер-ной деятельности   1.2.1 Поступления от аренды помещений              Алифанов Роман Игоревич            2.00   
                                                                                        ВЫМПЕЛКОМ ПАО                             
                                                                                        ПРОМОСФЕРА ООО                            
                                                                                        ТТК-СВЯЗЬ ООО                             
                                     1.2.3 Поступления от прочей аренды                 ВЫМПЕЛКОМ ПАО                     34.18   
...                                                                                                                         ...   
3. Приток от инвест-ной деятельности 3.1.3 Транспортные средства (Выручка от продажи... Филлипенков Антон Андреевич ЧЛ            
                                                                                        ЦЕНТРУС 1520 ООО                          
                                                                                        Шикерханов Гаджибала Рафикович            
                                     3.1.4 Инженерные сети и оборудование (Выручка о... РЕГИОН ПЛЮС ООО                           
All                                                                                                                    80419.14   

                                                                                                                                  \
Год                                                        

In [118]:
# pt_f1 = pd.pivot_table(df, index = ['Статья'], values = ['ФАКТ'], aggfunc=np.sum, fill_value='', margins=False)

pt_f1 = pd.pivot_table(df, index = ['Группа'], values = ['Факт'], aggfunc=np.sum, fill_value='', margins=False)

pt_f1

,Факт
Группа,
1. Приток от опер-ной деятельности,750611.48
2.1 Расходы на персонал,-194927.23
2.2 Материалы,-69414.57
2.3 Работы и услуги,-151062.41
2.4 Страхование,-1278.57
"2.5 Налоги, сборы и платежи",-118723.83
2.6 Прочие платежи,-107347.69
3. Приток от инвест-ной деятельности,90296.90
4. Отток от инвест-ной деятельности,-156647.66


In [119]:
pt_f2 = pd.pivot_table(df, index = ['Группа'], values = ['Факт'], columns='YM', aggfunc=np.sum, fill_value='', margins=False)

pt_f2

Факт                                \
YM                                     2021-01   2021-02   2021-03   2021-04   
Группа                                                                         
1. Приток от опер-ной деятельности    25463.82  67131.00  87512.77  84189.05   
2.1 Расходы на персонал              -13981.78 -18359.88 -23099.06 -25230.26   
2.2 Материалы                        -10496.98  -7670.10  -9957.80  -8021.01   
2.3 Работы и услуги                   -1216.59  -4102.68 -11813.46  -7293.61   
2.4 Страхование                                            -185.61   -320.17   
2.5 Налоги, сборы и платежи           -2152.70  -1637.33 -39037.98 -15288.22   
2.6 Прочие платежи                    -6213.31 -10287.82  -7475.40  -9346.28   
3. Приток от инвест-ной деятельности   3580.78  13288.14  21950.30  13580.50   
4. Отток от инвест-ной деятельности    -430.83  -7360.48 -12981.23 -33437.64   
5. Приток от фин-ной деятельности     17012.63             7000.00  50000.00   
6. Отток от фин-ной деятельности      -2500.00  -4036.57 -22220.97  -4611.77   
7. Внутренние перемещения                 0.00      0.00      0.00      0.00   
9. Возврат                                                   75.00             

                                                                              \
YM                                     2021-05   2021-06   2021-07   2021-08   
Группа                                                                         
1. Приток от опер-ной деятельности    54991.34  99807.97  66941.94  69698.56   
2.1 Расходы на персонал              -17242.91 -19609.19 -18166.57 -15846.63   
2.2 Материалы                         -4272.70  -4738.89  -3935.35  -4998.96   
2.3 Работы и услуги                   -5649.86 -11887.05 -10938.56 -27124.49   
2.4 Страхование                        -143.27   -282.63   -186.64    -58.44   
2.5 Налоги, сборы и платежи           -5920.17  -5995.15  -8812.03  -8935.36   
2.6 Прочие платежи                    -7873.66 -16567.82  -8999.66 -11530.45   
3. Приток от инвест-ной деятельности   7713.63  16801.95   2189.60    690.00   
4. Отток от инвест-ной деятельности  -10641.48 -10688.61 -18785.10  -6065.57   
5. Приток от фин-ной деятельности               50011.99                       
6. Отток от фин-ной деятельности      -1683.67 -70032.61  -8084.25  -7504.08   
7. Внутренние перемещения                 0.00      0.00                0.00   
9. Возврат                                        -22.18               22.82   

                                                                            
YM                                     2021-09   2021-10   2021-11 2021-12  
Группа                                                                      
1. Приток от опер-ной деятельности    95666.35  50863.14  48345.54          
2.1 Расходы на персонал              -14586.13 -15236.57 -13568.25          
2.2 Материалы                         -4829.13  -5264.12  -5229.53          
2.3 Работы и услуги                  -12340.57 -36040.77 -22654.77          
2.4 Страхование                                            -101.81          
2.5 Налоги, сборы и платежи           -8653.86 -14456.67  -7834.36          
2.6 Прочие платежи                    -8716.11  -9077.70 -11257.48   -2.00  
3. Приток от инвест-ной деятельности   6412.00   2730.00   1360.00          
4. Отток от инвест-ной деятельности  -19532.64 -16886.45 -19837.63          
5. Приток от фин-ной деятельности                                  1169.53  
6. Отток от фин-ной деятельности      -8651.38 -11382.65 -23919.41 9909.17  
7. Внутренние перемещения                 0.00      0.00      0.00          
9. Возврат                                7.31               73.03

In [129]:
#Сегодня
now = datetime.today()

#now.month
#now.day
#now.hour
#now.minute

# текущий день
td = now.day

# текущий месяц
tm = now.month

tm = 11 # 2021-12-02 пока поставил ноябрь 11

In [130]:
#Делаю табл только за текущий месяц

dfm = df[df["Месяц"] == tm] # tm


In [131]:
#Делаю табл притоку ДС по опер и инвест деят-ти только за текущий месяц

df_prm = df_pr[df_pr["Месяц"] == tm] # tm

In [132]:
df_prm.tail()

,date,YM,Год,Месяц,День,SD,SK,Группа,Статья,Факт,SKD1,SKD2,SKD3,SKK1,SKK2,SKK3
169717,2021-11-29,2021-11,2021,11,29,51,62.02,1. Приток от опер-ной деятельности,1.4.1 Продажа ТМЦ,1000.00,"40702810800020001286, ПАО СБЕРБАНК ООО ""ПРОМТЕХ""",1.4.1\tПродажа ТМЦ,,ВТОРМЕТАЛЛ ООО,Поставка металлолома № 17/201 от 20.04.2021,
169847,2021-11-30,2021-11,2021,11,30,51,62.01,1. Приток от опер-ной деятельности,1.4.1 Продажа ТМЦ,11.20,"40702810800020001286, ПАО СБЕРБАНК ООО ""ПРОМТЕХ""",1.4.1\tПродажа ТМЦ,,Агарков Александр Юрьевич,Продажа оросительных насадок б/у № 17/374 от 2...,
169848,2021-11-30,2021-11,2021,11,30,51,62.02,1. Приток от опер-ной деятельности,1.4.1 Продажа ТМЦ,16.00,"40702810800020001286, ПАО СБЕРБАНК ООО ""ПРОМТЕХ""",1.4.1\tПродажа ТМЦ,,Пугачева Ольга Владимировна,Продажа станка № 17/358 от 09.11.2021,
169849,2021-11-30,2021-11,2021,11,30,51,62.02,1. Приток от опер-ной деятельности,1.4.1 Продажа ТМЦ,50.00,"40702810800020001286, ПАО СБЕРБАНК ООО ""ПРОМТЕХ""",1.4.1\tПродажа ТМЦ,,Пугачева Ольга Владимировна,Продажа станка № 17/358 от 09.11.2021,
169850,2021-11-30,2021-11,2021,11,30,51,62.02,1. Приток от опер-ной деятельности,1.4.1 Продажа ТМЦ,1000.00,"40702810800020001286, ПАО СБЕРБАНК ООО ""ПРОМТЕХ""",1.4.1\tПродажа ТМЦ,,ВТОРМЕТАЛЛ ООО,Поставка металлолома № 17/201 от 20.04.2021,


In [133]:
# сводная табл по ОДДС по Статьям и Группам за текущий месяц

pt4 = pd.pivot_table(dfm, index = ['Группа', 'Статья'], values = ['Факт'], columns = ['Год', 'Месяц', 'День'], aggfunc=np.sum, fill_value='', margins=True)

pt4

Факт  \
Год                                                                                      2021   
Месяц                                                                                      11   
День                                                                                        1   
Группа                             Статья                                                       
1. Приток от опер-ной деятельности 1.2.1 Поступления от аренды помещений                        
                                   1.2.3 Поступления от прочей аренды                    3.05   
                                   1.3.3 Реализация услуг железнодорожного транспорта           
                                   1.3.7 Реализация услуг по водоснабжению                      
                                   1.4.1 Продажа ТМЦ                                  1000.00   
...                                                                                       ...   
6. Отток от фин-ной деятельности   6.03 Предоставление займов                                   
                                   6.10.1.4 Транспортные средства (Финансовая арен...           
7. Внутренние перемещения          7.1 Внутренние перемещения по расчетному счету               
9. Возврат                         Возврат денежных средств от поставщика                       
All                                                                                    995.71   

                                                                                               \
Год                                                                                             
Месяц                                                                                           
День                                                                                        2   
Группа                             Статья                                                       
1. Приток от опер-ной деятельности 1.2.1 Поступления от аренды помещений                        
                                   1.2.3 Поступления от прочей аренды                           
                                   1.3.3 Реализация услуг железнодорожного транспорта           
                                   1.3.7 Реализация услуг по водоснабжению              23.99   
                                   1.4.1 Продажа ТМЦ                                  2100.00   
...                                                                                       ...   
6. Отток от фин-ной деятельности   6.03 Предоставление займов                                   
                                   6.10.1.4 Транспортные средства (Финансовая арен...           
7. Внутренние перемещения          7.1 Внутренние перемещения по расчетному счету               
9. Возврат                         Возврат денежных средств от поставщика                       
All                                                                                   2123.99   

                                                                                               \
Год                                                                                             
Месяц                                                                                           
День                                                                                        3   
Группа                             Статья                                                       
1. Приток от опер-ной деятельности 1.2.1 Поступления от аренды помещений                        
                                   1.2.3 Поступления от прочей аренды                           
                                   1.3.3 Реализация услуг железнодорожного транспорта           
                                   1.3.7 Реализация услуг по водоснабжению                      
                                   1.4.1 Продажа ТМЦ                                  5000.00   
...          

In [134]:
# сводная табл по притоку ДС по опер и иныест деят по Статьям, Группам и контрагекнтам

pt5 = pd.pivot_table(df_prm, index = ['Группа', 'Статья', 'SKK1'], values = ['Факт'], columns = ['Год', 'Месяц', 'День'], aggfunc=np.sum, fill_value='', margins=True)

pt5

Факт  \
Год                                                                                                                                           2021   
Месяц                                                                                                                                           11   
День                                                                                                                                             1   
Группа                               Статья                                             SKK1                                                         
1. Приток от опер-ной деятельности   1.2.1 Поступления от аренды помещений              Алифанов Роман Игоревич                                      
                                                                                        ВЫМПЕЛКОМ ПАО                                                
                                     1.2.3 Поступления от прочей аренды                 ВЫМПЕЛКОМ ПАО                                                
                                                                                        ЭР-ТЕЛЕКОМ ХОЛДИНГ АО                                 3.05   
                                     1.3.3 Реализация услуг железнодорожного транспорта НЕФТЕТРАНССЕРВИС АО                                          
                                                                                        РЕД ХОЛТ ООО                                                 
                                                                                        РОСПЛАСТ ООО                                                 
                                                                                        ТРАНС-ЭКСПО ООО                                              
                                     1.3.7 Реализация услуг по водоснабжению            КОНЦЕССИИ ВОДОСНАБЖЕНИЯ ООО                                  
                                     1.4.1 Продажа ТМЦ                                  Агарев Алексей Юрьевич                                       
                                                                                        Агарков Александр Юрьевич                                    
                                                                                        Басангов Джангар Сергеевич                                   
                                                                                        ВТОРМЕТАЛЛ ООО                                     1000.00   
                                                                                        Жуков Николай Юрьевич                                        
                                                                                        Канатов Сергей Кайрулович                                    
                                                                                        Муслимов Ливат Михайлович                                    
                                                                                        Оркин Олег Иванович                                          
                                                                                        ПРОМОСФЕРА ООО                                               
                                                                                        Пугачева Ольга Владимировна                                  
                                                                                        РЕГИОН ПЛЮС ООО                                              
                                                                                        САРАТОВМЕТАЛЛ ООО ПКФ                                        
                                                                                        Самко Татьяна Сергеевна                                      
                                                                                        ТЕХН

In [135]:
# загрузка бюджета
# df_b = pd.read_excel('X:\\ФинДир\\Base\\pt_bdds21.xlsx', sheet_name='base')

#полный путь
# df_b = pd.read_excel('\\\\VPROMFS01\\Share\\ФинДир\\Base\\pt_bdds21_v2.xlsx', sheet_name='base')

# \\172.16.50.6\Smolenyshev_OV\Уровень_1\Base

df_b = pd.read_excel('\\\\172.25.0.10\\Share\\ФинДир\\Смоленышев\\Уровень_1\\Base\\pt_bdds21_v2.xlsx', sheet_name='base')

In [136]:
df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Код        730 non-null    float64       
 1   Раздел ДС  754 non-null    object        
 2   Статья     754 non-null    object        
 3   Период     754 non-null    datetime64[ns]
 4   Значение   754 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 29.6+ KB


In [137]:
# Отбор только по периоду

# df_b = df_b[(df_b['Период']>= '2021-01-01') & (df_b['Период']<= '2021-08-31')]

df_b = df_b[(df_b['Период']>= '2021-01-01') & (df_b['Период']<= dem)]

In [138]:
df_b.tail()

,Код,Раздел ДС,Статья,Период,Значение
749,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-08-01,17475.31
750,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-09-01,17688.00
751,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-10-01,18665.69
752,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-11-01,12039.36
753,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-12-01,6631.63


In [139]:
#  новый столбец с названием статей. В бюджете отток по инвест деят без цифр кода в начале строки, поэтому беру из столбца Код

df_b['Группа'] = ''

df_b.loc[df_b['Статья'].str[:1]=='1', 'Группа'] = '1. Приток от опер-ной деятельности'
df_b.loc[df_b['Статья'].str[:3]=='2.1', 'Группа'] = '2.1 Расходы на персонал' 
df_b.loc[df_b['Статья'].str[:3]=='2.2', 'Группа'] = '2.2 Материалы'
df_b.loc[df_b['Статья'].str[:3]=='2.3', 'Группа'] = '2.3 Работы и услуги'
df_b.loc[df_b['Статья'].str[:3]=='2.4', 'Группа'] = '2.4 Страхование'
df_b.loc[df_b['Статья'].str[:3]=='2.5', 'Группа'] = '2.5 Налоги, сборы и платежи'
df_b.loc[df_b['Статья'].str[:3]=='2.6', 'Группа'] = '2.6 Прочие платежи'
df_b.loc[df_b['Статья'].str[:1]=='3', 'Группа'] = '3. Приток от инвест-ной деятельности'
df_b.loc[df_b['Статья'].str[:1]=='4', 'Группа'] = '4. Отток от инвест-ной деятельности'
df_b.loc[df_b['Статья']=='Метанол', 'Группа'] = '4. Отток от инвест-ной деятельности'
df_b.loc[df_b['Статья']=='ОЭС', 'Группа'] = '4. Отток от инвест-ной деятельности'
df_b.loc[df_b['Статья']=='Инфраструткрура площадки', 'Группа'] = '4. Отток от инвест-ной деятельности'
df_b.loc[df_b['Статья']=='Выкуп земли', 'Группа'] = '4. Отток от инвест-ной деятельности'
df_b.loc[df_b['Статья'].str[:1]=='5', 'Группа'] = '5. Приток от фин-ной деятельности'
df_b.loc[df_b['Статья'].str[:1]=='6', 'Группа'] = '6. Отток от фин-ной деятельности'
df_b.loc[df_b['Статья'].str[:1]=='7', 'Группа'] = '7. Внутренние перемещения'
df_b.loc[df_b['Статья'].str[:7]=='Возврат', 'Группа'] = '9. Возврат'

In [140]:
# преобразование в число
df_b['БЮДЖЕТ'] = pd.to_numeric(df_b['Значение']).round(2)

In [141]:
df_b['Группа'].str[:1].unique()

array(['', '1', '2', '3', '4', '5', '6'], dtype=object)

In [142]:
# Create new column using conditional indexing
df_b["План"] = ""
df_b.loc[df_b['Группа'].str[:1] == '1', "План"] = df_b['БЮДЖЕТ']
df_b.loc[df_b['Группа'].str[:1] == '3', "План"] = df_b['БЮДЖЕТ']
df_b.loc[df_b['Группа'].str[:1] == '5', "План"] = df_b['БЮДЖЕТ']
df_b.loc[df_b['Группа'].str[:1] == '2', "План"] = df_b['БЮДЖЕТ']*-1
df_b.loc[df_b['Группа'].str[:1] == '4', "План"] = df_b['БЮДЖЕТ']*-1
df_b.loc[df_b['Группа'].str[:1] == '6', "План"] = df_b['БЮДЖЕТ']*-1
df_b

,Код,Раздел ДС,Статья,Период,Значение,Группа,БЮДЖЕТ,План
0,0.00,Остаток ден. средств на нач пер,Остаток ден. средств на нач пер,2021-01-01,1804.38,,1804.38,
1,0.00,Остаток ден. средств на нач пер,Остаток ден. средств на нач пер,2021-02-01,10871.48,,10871.48,
2,0.00,Остаток ден. средств на нач пер,Остаток ден. средств на нач пер,2021-03-01,37786.48,,37786.48,
3,0.00,Остаток ден. средств на нач пер,Остаток ден. средств на нач пер,2021-04-01,3081.19,,3081.19,
4,0.00,Остаток ден. средств на нач пер,Остаток ден. средств на нач пер,2021-05-01,4770.64,,4770.64,
...,...,...,...,...,...,...,...,...
749,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-08-01,17475.31,,17475.31,
750,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-09-01,17688.00,,17688.00,
751,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-10-01,18665.69,,18665.69,
752,8.00,Остаток ден. средств на кон пер,Остаток ден. средств на кон пер,2021-11-01,12039.36,,12039.36,


In [143]:
df_b['План'] = pd.to_numeric(df_b['План']).round(2)

In [144]:
# переименовал столбец
df_b = df_b.rename(columns= {'Период': 'date'})

# df_b = df_b.rename(columns= {'Период': 'date', 'Значение': 'ST'})

In [145]:
# стролбцы с периодами
df_b['Год'] = df_b['date'].dt.year.astype('Int64').values
df_b['Месяц'] = df_b['date'].dt.month.astype('Int64').values
df_b['День'] = df_b['date'].dt.day.astype('Int64').values

In [146]:
#только год и месяц
df_b['YM'] = df_b['date'].dt.to_period('M')

In [147]:
# df_b.tail()
# df_b.head()
df_b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 754 entries, 0 to 753
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Код        730 non-null    float64       
 1   Раздел ДС  754 non-null    object        
 2   Статья     754 non-null    object        
 3   date       754 non-null    datetime64[ns]
 4   Значение   754 non-null    float64       
 5   Группа     754 non-null    object        
 6   БЮДЖЕТ     754 non-null    float64       
 7   План       572 non-null    float64       
 8   Год        754 non-null    Int64         
 9   Месяц      754 non-null    Int64         
 10  День       754 non-null    Int64         
 11  YM         754 non-null    period[M]     
dtypes: Int64(3), datetime64[ns](1), float64(4), object(3), period[M](1)
memory usage: 95.0+ KB


In [148]:
# тестовая сводная табл по БДДС

pt_b0 = pd.pivot_table(df_b, index = ['Группа'], values = ['БЮДЖЕТ'], columns = ['Год', 'Месяц'], aggfunc=np.sum, fill_value='', margins=True)

pt_b0

БЮДЖЕТ                                \
Год                                       2021                                 
Месяц                                        1         2         3         4   
Группа                                                                         
                                     197105.06 370333.96 501811.04 420640.91   
1. Приток от опер-ной деятельности    25524.60  79054.00  97635.84  76879.27   
2.1 Расходы на персонал               13981.70  18414.00  20894.24  21538.80   
2.2 Материалы                         10497.05   7670.00   6708.76   5214.59   
2.3 Работы и услуги                    1216.55   4103.00   8187.90  11802.53   
2.4 Страхование                                             850.00      0.00   
2.5 Налоги, сборы и платежи            2152.50   1621.00  31227.00  14090.00   
2.6 Прочие платежи                     6211.10  10298.00   8868.00   9368.00   
3. Приток от инвест-ной деятельности   3519.80   1365.00  17600.00  26318.00   
4. Отток от инвест-ной деятельности     431.30   7361.00  52980.24  37042.38   
5. Приток от фин-ной деятельности     17012.90                0.00      0.00   
6. Отток от фин-ной деятельности       2500.00   4037.00  20225.00   2451.51   
All                                  280152.56 504256.96 766988.02 625345.99   

                                                                              \
Год                                                                            
Месяц                                        5         6         7         8   
Группа                                                                         
                                     496340.75 468968.74 383163.55 352448.56   
1. Приток от опер-ной деятельности    70685.47  49782.55  59740.47  48820.47   
2.1 Расходы на персонал               19640.43  18550.74  17917.05  18008.40   
2.2 Материалы                          4944.79   4645.30   4579.18   4983.39   
2.3 Работы и услуги                   18148.15  20488.15  18064.30  15365.10   
2.4 Страхование                           0.00     50.00      0.00      0.00   
2.5 Налоги, сборы и платежи            7700.00   5500.00   5840.00   5450.00   
2.6 Прочие платежи                    10628.00  10628.00  16038.00  10678.00   
3. Приток от инвест-ной деятельности   5675.00   4100.00   3500.00   3200.00   
4. Отток от инвест-ной деятельности   30218.14  41486.00  23585.00  23540.00   
5. Приток от фин-ной деятельности     40000.00  50000.00  20000.00  26000.00   
6. Отток от фин-ной деятельности       3723.35   1352.29   1635.85   5411.66   
All                                  707704.08 675551.77 554063.40 513905.58   

                                                                              \
Год                                                                            
Месяц                                        9        10        11        12   
Группа                                                                         
                                     386338.31 293354.61 290074.92 253100.24   
1. Приток от опер-ной деятельности    39693.75  31750.23  32842.47  26607.31   
2.1 Расходы на персонал               18008.40  18133.68  18133.68  18033.68   
2.2 Материалы                          5559.64   6348.27   7255.31   7739.51   
2.3 Работы и услуги                   11226.60   4330.35   3705.95   2991.95   
2.4 Страхование                           0.00      0.00      0.00      0.00   
2.5 Налоги, сборы и платежи            3950.00   4990.00   4600.00   4600.00   
2.6 Прочие платежи                    10678.00   9368.00   9858.00   8868.00   
3. Приток от инвест-ной деятельности   3100.00   2500.00   2000.00   2000.00   
4. Отток от инвест-ной деятельности   32454.00  17115.00  24755.00  18345.00   
5. Приток от фин-ной деятельности     45000.00  30000.00  30000.00  30000.00   
6. Отток от фин-ной деятельности       5704.44   2987.23   3160.87   3436.91   
All                                  5

In [149]:
# pt_b1 = pd.pivot_table(df_b, index = ['Статья'], values = ['БЮДЖЕТ'], aggfunc=np.sum, fill_value='', margins=False)

pt_b1 = pd.pivot_table(df_b, index = ['Группа'], values = ['План'], aggfunc=np.sum, fill_value='', margins=False)

pt_b1

,План
Группа,
,0.00
1. Приток от опер-ной деятельности,639016.43
2.1 Расходы на персонал,-221254.80
2.2 Материалы,-76145.79
2.3 Работы и услуги,-119630.53
2.4 Страхование,-900.00
"2.5 Налоги, сборы и платежи",-91720.50
2.6 Прочие платежи,-121489.10
3. Приток от инвест-ной деятельности,74877.80


In [150]:
pt_b2 = pd.pivot_table(df_b, index = ['Группа'], values = ['План'], columns = ['YM'], aggfunc=np.sum, fill_value='', margins=False)

pt_b2

План                                \
YM                                     2021-01   2021-02   2021-03   2021-04   
Группа                                                                         
                                          0.00      0.00      0.00      0.00   
1. Приток от опер-ной деятельности    25524.60  79054.00  97635.84  76879.27   
2.1 Расходы на персонал              -13981.70 -18414.00 -20894.24 -21538.80   
2.2 Материалы                        -10497.05  -7670.00  -6708.76  -5214.59   
2.3 Работы и услуги                   -1216.55  -4103.00  -8187.90 -11802.53   
2.4 Страхование                                            -850.00      0.00   
2.5 Налоги, сборы и платежи           -2152.50  -1621.00 -31227.00 -14090.00   
2.6 Прочие платежи                    -6211.10 -10298.00  -8868.00  -9368.00   
3. Приток от инвест-ной деятельности   3519.80   1365.00  17600.00  26318.00   
4. Отток от инвест-ной деятельности    -431.30  -7361.00 -52980.24 -37042.38   
5. Приток от фин-ной деятельности     17012.90                0.00      0.00   
6. Отток от фин-ной деятельности      -2500.00  -4037.00 -20225.00  -2451.51   

                                                                              \
YM                                     2021-05   2021-06   2021-07   2021-08   
Группа                                                                         
                                          0.00      0.00      0.00      0.00   
1. Приток от опер-ной деятельности    70685.47  49782.55  59740.47  48820.47   
2.1 Расходы на персонал              -19640.43 -18550.74 -17917.05 -18008.40   
2.2 Материалы                         -4944.79  -4645.30  -4579.18  -4983.39   
2.3 Работы и услуги                  -18148.15 -20488.15 -18064.30 -15365.10   
2.4 Страхование                           0.00    -50.00      0.00      0.00   
2.5 Налоги, сборы и платежи           -7700.00  -5500.00  -5840.00  -5450.00   
2.6 Прочие платежи                   -10628.00 -10628.00 -16038.00 -10678.00   
3. Приток от инвест-ной деятельности   5675.00   4100.00   3500.00   3200.00   
4. Отток от инвест-ной деятельности  -30218.14 -41486.00 -23585.00 -23540.00   
5. Приток от фин-ной деятельности     40000.00  50000.00  20000.00  26000.00   
6. Отток от фин-ной деятельности      -3723.35  -1352.29  -1635.85  -5411.66   

                                                                              
YM                                     2021-09   2021-10   2021-11   2021-12  
Группа                                                                        
                                          0.00      0.00      0.00      0.00  
1. Приток от опер-ной деятельности    39693.75  31750.23  32842.47  26607.31  
2.1 Расходы на персонал              -18008.40 -18133.68 -18133.68 -18033.68  
2.2 Материалы                         -5559.64  -6348.27  -7255.31  -7739.51  
2.3 Работы и услуги                  -11226.60  -4330.35  -3705.95  -2991.95  
2.4 Страхование                           0.00      0.00      0.00      0.00  
2.5 Налоги, сборы и платежи           -3950.00  -4990.00  -4600.00  -4600.00  
2.6 Прочие платежи                   -10678.00  -9368.00  -9858.00  -8868.00  
3. Приток от инвест-ной деятельности   3100.00   2500.00   2000.00   2000.00  
4. Отток от инвест-ной деятельности  -32454.00 -17115.00 -24755.00 -18345.00  
5. Приток от фин-ной деятельности     45000.00  30000.00  30000.00  30000.00  
6. Отток от фин-ной деятельности      -5704.44  -2987.23  -3160.87  -3436.91

In [151]:
pt_f1 = pt_f1.reset_index()
pt_b1 = pt_b1.reset_index()

In [152]:
pt_f1
# pt_b1

,Группа,Факт
0,1. Приток от опер-ной деятельности,750611.48
1,2.1 Расходы на персонал,-194927.23
2,2.2 Материалы,-69414.57
3,2.3 Работы и услуги,-151062.41
4,2.4 Страхование,-1278.57
5,"2.5 Налоги, сборы и платежи",-118723.83
6,2.6 Прочие платежи,-107347.69
7,3. Приток от инвест-ной деятельности,90296.90
8,4. Отток от инвест-ной деятельности,-156647.66
9,5. Приток от фин-ной деятельности,125194.15


In [153]:
m1 = pt_b1.merge(pt_f1, how = 'outer')
m1  # это уже на много лучше. Неявноя слияние по всем общим столбцам

# m1.reset_index()

m1

,Группа,План,Факт
0,,0.00,NaN
1,1. Приток от опер-ной деятельности,639016.43,750611.48
2,2.1 Расходы на персонал,-221254.80,-194927.23
3,2.2 Материалы,-76145.79,-69414.57
4,2.3 Работы и услуги,-119630.53,-151062.41
5,2.4 Страхование,-900.00,-1278.57
6,"2.5 Налоги, сборы и платежи",-91720.50,-118723.83
7,2.6 Прочие платежи,-121489.10,-107347.69
8,3. Приток от инвест-ной деятельности,74877.80,90296.90
9,4. Отток от инвест-ной деятельности,-309313.06,-156647.66


In [154]:
m1 = m1.set_index('Группа')

m1

,План,Факт
Группа,,
,0.00,NaN
1. Приток от опер-ной деятельности,639016.43,750611.48
2.1 Расходы на персонал,-221254.80,-194927.23
2.2 Материалы,-76145.79,-69414.57
2.3 Работы и услуги,-119630.53,-151062.41
2.4 Страхование,-900.00,-1278.57
"2.5 Налоги, сборы и платежи",-91720.50,-118723.83
2.6 Прочие платежи,-121489.10,-107347.69
3. Приток от инвест-ной деятельности,74877.80,90296.90


In [155]:
# добавляю столбцы
m1['абс.откл.'] = m1['Факт'] - m1['План']
m1['отн.откл.%'] = ((m1['абс.откл.'] / m1['План'])*100).round(2)

In [156]:
m1

,План,Факт,абс.откл.,отн.откл.%
Группа,,,,
,0.00,NaN,NaN,NaN
1. Приток от опер-ной деятельности,639016.43,750611.48,111595.05,17.46
2.1 Расходы на персонал,-221254.80,-194927.23,26327.57,-11.90
2.2 Материалы,-76145.79,-69414.57,6731.22,-8.84
2.3 Работы и услуги,-119630.53,-151062.41,-31431.88,26.27
2.4 Страхование,-900.00,-1278.57,-378.57,42.06
"2.5 Налоги, сборы и платежи",-91720.50,-118723.83,-27003.33,29.44
2.6 Прочие платежи,-121489.10,-107347.69,14141.41,-11.64
3. Приток от инвест-ной деятельности,74877.80,90296.90,15419.10,20.59


In [157]:
# удаляю и обрабатываю NaN


m2 = m1.fillna(0)

m2

,План,Факт,абс.откл.,отн.откл.%
Группа,,,,
,0.00,0.00,0.00,0.00
1. Приток от опер-ной деятельности,639016.43,750611.48,111595.05,17.46
2.1 Расходы на персонал,-221254.80,-194927.23,26327.57,-11.90
2.2 Материалы,-76145.79,-69414.57,6731.22,-8.84
2.3 Работы и услуги,-119630.53,-151062.41,-31431.88,26.27
2.4 Страхование,-900.00,-1278.57,-378.57,42.06
"2.5 Налоги, сборы и платежи",-91720.50,-118723.83,-27003.33,29.44
2.6 Прочие платежи,-121489.10,-107347.69,14141.41,-11.64
3. Приток от инвест-ной деятельности,74877.80,90296.90,15419.10,20.59


In [158]:
# переменные с итогами
plan = m1['План'].sum().round(1)
fact = m1['Факт'].sum().round(1)
abs = (fact - plan).round(1)
otn = ((abs/plan)*100).round(2)

print(plan, fact, abs, otn)

4827.2 12138.4 7311.2 151.46


In [159]:
m0 = pt_b1.merge(pt_f1, how = 'outer')

m0 = m0.fillna(0)

In [160]:
# m0.info()

In [161]:
# текущий месяц
tm = today.month
ty = today.year

print(tm)
print(ty)


12
2021


In [162]:
# df['Группа'].unique()

In [163]:
# приток только тек месяц опет деят
df1 = df[(df["Месяц"] == tm) & (df['Группа'].str[:1]=='1')]

df1.tail()

,date,YM,Год,Месяц,День,SD,SK,Группа,Статья,Факт,SKD1,SKD2,SKD3,SKK1,SKK2,SKK3


In [164]:
# приток по опер деят только за тек мес
pritok_tm = df1['Факт'].sum().round(1)

pritok_tm

0.0

In [165]:
# отток только тек месяц опет деят
df2 = df[(df["Месяц"] == tm) & (df['Группа'].str[:1]=='2')]

df2.tail()

,date,YM,Год,Месяц,День,SD,SK,Группа,Статья,Факт,SKD1,SKD2,SKD3,SKK1,SKK2,SKK3
169932,2021-12-01,2021-12,2021,12,1,91.02,51,2.6 Прочие платежи,2.6.13 Банковские услуги,-2.00,91.2.01 Услуги банков,,,"40702810800020001286, ПАО СБЕРБАНК ООО ""ПРОМТЕХ""",2.6.13 Банковские услуги,


In [166]:
# приток по опер деят только за тек мес
ottok_tm = df2['Факт'].sum().round(1)

ottok_tm

-2.0

In [167]:
m3 = pt_b2.merge(pt_f2, how = 'outer', on='Группа')
# m3.reset_index()

m3.fillna(0)



План                                \
YM                                     2021-01   2021-02   2021-03   2021-04   
Группа                                                                         
                                          0.00      0.00      0.00      0.00   
1. Приток от опер-ной деятельности    25524.60  79054.00  97635.84  76879.27   
2.1 Расходы на персонал              -13981.70 -18414.00 -20894.24 -21538.80   
2.2 Материалы                        -10497.05  -7670.00  -6708.76  -5214.59   
2.3 Работы и услуги                   -1216.55  -4103.00  -8187.90 -11802.53   
2.4 Страхование                                            -850.00      0.00   
2.5 Налоги, сборы и платежи           -2152.50  -1621.00 -31227.00 -14090.00   
2.6 Прочие платежи                    -6211.10 -10298.00  -8868.00  -9368.00   
3. Приток от инвест-ной деятельности   3519.80   1365.00  17600.00  26318.00   
4. Отток от инвест-ной деятельности    -431.30  -7361.00 -52980.24 -37042.38   
5. Приток от фин-ной деятельности     17012.90                0.00      0.00   
6. Отток от фин-ной деятельности      -2500.00  -4037.00 -20225.00  -2451.51   
7. Внутренние перемещения                    0         0      0.00      0.00   
9. Возврат                                   0         0      0.00      0.00   

                                                                              \
YM                                     2021-05   2021-06   2021-07   2021-08   
Группа                                                                         
                                          0.00      0.00      0.00      0.00   
1. Приток от опер-ной деятельности    70685.47  49782.55  59740.47  48820.47   
2.1 Расходы на персонал              -19640.43 -18550.74 -17917.05 -18008.40   
2.2 Материалы                         -4944.79  -4645.30  -4579.18  -4983.39   
2.3 Работы и услуги                  -18148.15 -20488.15 -18064.30 -15365.10   
2.4 Страхование                           0.00    -50.00      0.00      0.00   
2.5 Налоги, сборы и платежи           -7700.00  -5500.00  -5840.00  -5450.00   
2.6 Прочие платежи                   -10628.00 -10628.00 -16038.00 -10678.00   
3. Приток от инвест-ной деятельности   5675.00   4100.00   3500.00   3200.00   
4. Отток от инвест-ной деятельности  -30218.14 -41486.00 -23585.00 -23540.00   
5. Приток от фин-ной деятельности     40000.00  50000.00  20000.00  26000.00   
6. Отток от фин-ной деятельности      -3723.35  -1352.29  -1635.85  -5411.66   
7. Внутренние перемещения                 0.00      0.00      0.00      0.00   
9. Возврат                                0.00      0.00      0.00      0.00   

                                                          ...      Факт  \
YM                                     2021-09   2021-10  ...   2021-03   
Группа                                                    ...             
                                          0.00      0.00  ...      0.00   
1. Приток от опер-ной деятельности    39693.75  31750.23  ...  87512.77   
2.1 Расходы на персонал              -18008.40 -18133.68  ... -23099.06   
2.2 Материалы                         -5559.64  -6348.27  ...  -9957.80   
2.3 Работы и услуги                  -11226.60  -4330.35  ... -11813.46   
2.4 Страхование                           0.00      0.00  ...   -185.61   
2.5 Налоги, сборы и платежи           -3950.00  -4990.00  ... -39037.98   
2.6 Прочие платежи                   -10678.00  -9368.00  ...  -7475.40   
3. Приток от инвест-ной деятельности   3100.00   2500.00  ...  21950.30   
4. Отток от инвест-ной деятельности  -32454.00 -17115.00  ... -12981.23   
5. Приток от фин-ной деятельности     45000.00  30000.00  ...   7000.00   
6. Отток от фин-ной деятельности      -5704.44  -2987.23  ... -22220.97   
7. Внутренние перемещения                 0.00      0.00  ...      0.00   
9. Возврат                                0.00      0.00  ...     75.00   

                                            

In [168]:
def write_to_html_file_2(df, title='', filename='out.html'):
    '''
    Write an entire dataframe to an HTML file with nice formatting.
    '''

    result = '''
<html>
<head>
<style>

    h2 {
    text-align: center;
    font-family: Helvetica, Arial, sans-serif;
    color: #1919bc;
  }
  table { 
    margin-left: auto;
    margin-right: auto;
  }
  table, th, td {
    border: 0.5px solid black;
    border-collapse: collapse;
    border-color: #1919bc;
  }
  th, td {
    padding: 5px;
    font-family: Helvetica, Arial, sans-serif;
    font-size: 80%;
    text-align: left;
    color: #1919bc;
    vertical-align: middle;
  }
  table tbody tr:hover {
    background-color: #dddddd;
  }
  .wide {
    width: 90%;
  }

  thead > tr:nth-child(2) th:not(:first-child) {
    text-align: center;
  }
  tbody td {
    text-align: right;
  }



</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    result += df.to_html(escape=False) #classes='wide', float_format='{:.2f}'.format
    result += '''

<script>
  function numberWithSpaces(x) {
    var rounded = Math.floor(Number(x));
    var parts = rounded.toString().split(".");
    parts[0] = parts[0].replace(/\B(?=(\d{3})+(?!\d))/g, " ");
    return parts.join(".");
  }

  var items = document.querySelectorAll('tbody td');

  items.forEach(function(item) {
    var formattedNumber = numberWithSpaces(item.textContent);
    if(item.textContent === '') {
      return;
    }
    item.textContent = formattedNumber;
  });
</script>


</body>

<center>
	<footer>
      <br />
      <br />
      <b>Создал:</b> <i>Смоленышев Олег В.</i>
      <br />
      <a href="mailto:smolenyshev@gtm-one.com">smolenyshev@gtm-one.com</a>
  </footer>
</center>


</html>
'''
    with open(filename, 'w') as f:
        f.write(result)

In [169]:
txt_today = f"Отчет создан: {today2}"

txt_today

'Отчет создан: 2021-12-02 16:15'

In [170]:
# 2021 10 22

def write_to_html_file_3(df, title='', footer='', filename='out.html'):
    '''
    Write an entire dataframe to an HTML file with nice formatting.
    '''

    result = '''
<html>
<head>
<style>

    h2 {
    text-align: center;
    font-family: Helvetica, Arial, sans-serif;
    color: #1919bc;
  }
  table { 
    margin-left: auto;
    margin-right: auto;
  }
  table, th, td {
    border: 0.5px solid black;
    border-collapse: collapse;
    border-color: #1919bc;
  }
  th, td {
    padding: 5px;
    font-family: Helvetica, Arial, sans-serif;
    font-size: 80%;
    text-align: left;
    color: #1919bc;
    vertical-align: middle;
  }
  table tbody tr:hover {
    background-color: #dddddd;
  }
  .wide {
    width: 90%;
  }

  thead > tr:nth-child(2) th:not(:first-child) {
    text-align: center;
  }
  tbody td {
    text-align: right;
  }



</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    result += df.to_html(escape=False) #classes='wide', float_format='{:.2f}'.format
    result += '''

<script>
  function numberWithSpaces(x) {
    var rounded = Math.floor(Number(x));
    var parts = rounded.toString().split(".");
    parts[0] = parts[0].replace(/\B(?=(\d{3})+(?!\d))/g, " ");
    return parts.join(".");
  }

  var items = document.querySelectorAll('tbody td');

  items.forEach(function(item) {
    var formattedNumber = numberWithSpaces(item.textContent);
    if(item.textContent === '') {
      return;
    }
    item.textContent = formattedNumber;
  });
</script>


</body>

<center>
	<footer>
      <br />
      <br />
      <b>Создал:</b> <i>Смоленышев Олег В.</i>
      <br />
      <a href="mailto:smolenyshev@gtm-one.com">smolenyshev@gtm-one.com</a>
      <br />
      '''
    result += '<i> %s </i>\n' % footer
    result += '''
  </footer>
</center>


</html>
'''
    with open(filename, 'w') as f:
        f.write(result)

In [171]:
#текст для заголовков с датой
txt1 = f"Промтех. ОДДС за 2021 год (тыс.руб.) по группам и статьям ДС. <br> Данные из 1С ERP.<br> Отчет на дату: {lastday2}"
txt2 = f"Промтех. ОДДС за 2021 год (тыс.руб.) только по группам ДС. <br>  Данные из 1С ERP.<br> Отчет на дату: {lastday2}."
txt3 = f"Промтех. Приток ден.средств по опер-ной и инвест-ной деятельности в разрезе контрагенов за 2021 год (тыс.руб.). <br> Данные из 1С ERP.<br> Отчет на дату: {lastday2}"
txt4 = f"Промтех. ОДДС за по дням за текущий ({tm}) месяц. (тыс.руб.). <br> Данные из 1С ERP.<br> Отчет на дату: {lastday2}"
txt5 = f"Промтех. Приток ден.средств по опер-ной и инвест-ной деятельности в разрезе контрагенов за текущий ({tm}) месяц. (тыс.руб.). <br> Данные из 1С ERP.<br> Отчет на дату: {lastday2}"

txt5

"Промтех. Приток ден.средств по опер-ной и инвест-ной деятельности в разрезе контрагенов за текущий (12) месяц. (тыс.руб.). <br> Данные из 1С ERP.<br> Отчет на дату: ['2021-12-01']"

In [172]:
text3 = f"Промтех. План-факт движения ден.средств в целом с января 2021 по текущий месяц. Факт из 1С ERP. \n План из бюджета представленного к защите 24 марта 2021. Актуализация на дату: {lastday2}. \n \
Всего чистый суммарный приток ден.средств по плану: {plan} тыс.руб., по факту: {fact} тыс.руб., абс.откл.: {abs} тыс.руб., отн.отклонение: {otn}% \n \
Приток ден средств по опер.деят-ти за текущий ({ty}-{tm}) месяц составил: {pritok_tm} тыс.руб. \n \
Отток ден средств по опер.деят-ти за текущий ({ty}-{tm}) месяц составил: {ottok_tm} тыс.руб."

print(text3)

Промтех. План-факт движения ден.средств в целом с января 2021 по текущий месяц. Факт из 1С ERP. 
 План из бюджета представленного к защите 24 марта 2021. Актуализация на дату: ['2021-12-01']. 
 Всего чистый суммарный приток ден.средств по плану: 4827.2 тыс.руб., по факту: 12138.4 тыс.руб., абс.откл.: 7311.2 тыс.руб., отн.отклонение: 151.46% 
 Приток ден средств по опер.деят-ти за текущий (2021-12) месяц составил: 0.0 тыс.руб. 
 Отток ден средств по опер.деят-ти за текущий (2021-12) месяц составил: -2.0 тыс.руб.


In [173]:
# создание html файлов
# Старый вывод:
# write_to_html_file_2(pt1, txt1, 'Промтех_ОДДС_2021.html')
# write_to_html_file_2(pt2, txt2, 'Промтех_ОДДС_2021_кратк.html')
# write_to_html_file_2(pt3, txt3, 'Промтех_ОДДС_приток_2021.html')
# write_to_html_file_2(pt4, txt4, 'Промтех_ОДДС_2021_текущ_мес.html')
# write_to_html_file_2(pt5, txt5, 'Промтех_ОДДС_приток_2021_текущ_мес.html')

# write_to_html_file_2(m2, text3, 'Промтех_ОДДС_план-факт_2021.html')


In [174]:
# 2021 10 22 

write_to_html_file_3(pt1, txt1, txt_today, 'Промтех_ОДДС_2021.html')
write_to_html_file_3(pt2, txt2, txt_today, 'Промтех_ОДДС_2021_кратк.html')
write_to_html_file_3(pt3, txt3, txt_today, 'Промтех_ОДДС_приток_2021.html')
write_to_html_file_3(pt4, txt4, txt_today, 'Промтех_ОДДС_2021_текущ_мес.html')
write_to_html_file_3(pt5, txt5, txt_today, 'Промтех_ОДДС_приток_2021_текущ_мес.html')

write_to_html_file_3(m2, text3, txt_today, 'Промтех_ОДДС_план-факт_2021.html')

In [175]:
# Отправка на почту

import smtplib
# import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

In [176]:
# Настройки

sender_email = 'syslog2@XXX.XXX'


receiver_email = 'smolenyshev@gtm-one.com'

username = 'XXX'
password = 'XXX'
server = smtplib.SMTP('XXX')
subject = f'Промтех. ОДДС. Отчет на дату: {lastday2}' #тема пиьма

# body = f"Добрый день.\n\nПередаю для инф. ОДДС. Отчет на дату: {lastday2}. \nОтчет сформирован на основании данных из 1С ERP. \
#          С 9 сентября 2021 отчет запускается каждый рабочий день в 08:10.\n С 21 октября 2021 отчеты изменены.\n\n \
#          С уважением, Смоленышев О.В."  #текст внутри пиьма, если нужен

body = f"Доброе утро. \n\n Передаю для информации ОДДС фактическое движение и план-факт.\n \
        Отчеты формируются на основании данных из 1С ERP и и представленнго к защите бюджета от 24 марта 2021. Отчет на дату: {lastday2}. \n \
        С 9 сентября 2021 отчет запускается каждый рабочий день в 08:10. \n\n \
        Всего чистый суммарный приток ден.средств по плану: {plan} тыс.руб., по факту: {fact} тыс.руб., \n \
        абс.откл.: {abs} тыс.руб., отн.отклонение: {otn}%  \n\n \
        Приток ден средств по опер.деят-ти за текущий ({ty}-{tm}) месяц составил: {pritok_tm} тыс.руб. \n \
        Отток ден средств по опер.деят-ти за текущий ({ty}-{tm}) месяц составил: {ottok_tm} тыс.руб. \n\n \
        С 21 октября 2021 отчеты адоптированы. \n\
        С уважением, Смоленышев О.В."


dir_path = "C:\\Reports"  #путь к файлам

files = ['Промтех_ОДДС_2021.html', 'Промтех_ОДДС_2021_кратк.html', 'Промтех_ОДДС_приток_2021.html', 'Промтех_ОДДС_2021_текущ_мес.html', 
        'Промтех_ОДДС_приток_2021_текущ_мес.html', 'Промтех_ОДДС_план-факт_2021.html']        #список отправляемых файлов. , 'cf_graf.html'

In [177]:
# Создание составного сообщения и установка заголовка

message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject

# Внесение тела письма
message.attach(MIMEText(body, "plain"))

for f in files:
    file_path = os.path.join(dir_path,f)
    attachment = MIMEApplication(open(file_path, "rb").read(), _subtype="txt")
    attachment.add_header('Content-Disposition', 'attachment', filename=f)
    message.attach(attachment)

# Отпавляем письмо

server.starttls()
server.ehlo()
server.login(username, password)
server.sendmail(message['From'], message['To'].split(','), message.as_string())
server.quit()
print(receiver_email, files, sep='\n')

smolenyshev@gtm-one.com
['Промтех_ОДДС_2021.html', 'Промтех_ОДДС_2021_кратк.html', 'Промтех_ОДДС_приток_2021.html', 'Промтех_ОДДС_2021_текущ_мес.html', 'Промтех_ОДДС_приток_2021_текущ_мес.html', 'Промтех_ОДДС_план-факт_2021.html']
